In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import join, abspath
from os.path import basename

In [2]:
markup_path = '.\markup'

In [3]:
def process_csv(file_path):
    data = pd.read_csv(file_path)
    tokens = data.token
    positions = data.pos
    sentences = data.sentence
    norms = data.norm
    
    current_sentence = sentences[0]
    lines = {'X': [], 'Y': []}    
    sent_tokens = []
    sent_norms = []
    for i in range(len(tokens)):    
        if sentences[i] == current_sentence:
            sent_tokens.append(tokens[i])
            sent_norms.append(norms[i])
        else:
            current_sentence = sentences[i]
            lines['X'].append(' '.join(sent_tokens))
            lines['Y'].append(' '.join(sent_norms))
            sent_tokens = []
            sent_norms = []
    else:
        lines['X'].append(' '.join(sent_tokens))
        lines['Y'].append(' '.join(sent_norms))
        
    return lines

In [4]:
raw_data = {'X': [], 'Y': []}
file_path = ''

for folder in listdir(markup_path)[:30]:
    folder_path = join(markup_path, folder)
    for data_file in listdir(folder_path):           
        file_path = join(folder_path, data_file)
        file_path = abspath(file_path)
        try:
            lines = process_csv(open(file_path, 'r', encoding='utf-8'))
            raw_data['X'].extend(lines['X'])
            raw_data['Y'].extend(lines['Y'])
        except Exception as e:
            print(basename(file_path))
            print(e)
    print(folder, end=' ')


( 1 2 3 4 5 6 7 8 9 A B C D E F G H I Java.csv
sequence item 16: expected str instance, float found
J K L Magi.csv
index out of bounds
M N O PHP.csv
sequence item 1: expected str instance, float found
P Q R S T 

In [5]:
print(raw_data['X'][0])
print(raw_data['Y'][0])

( 9154 ) Кольцово ( 9154 ) Кольцóво ( ) — астероид из группы главного пояса , открытый 16 сентября 1982 года Людмилой Черных в Крымской обсерватории и назван в честь посёлка Кольцово , расположенного неподалёку от Новосибирска .
( 9154 ) кольцово ( 9154 ) кольцóвый ( ) — астероид из группа главный пояс , открытый 16 сентябрь 1982 год людмила чёрный в крымский обсерватория и назвать в честь посёлок кольцово , расположить неподалёку от новосибирск .


In [6]:
print(len(raw_data['X']))
print(len(raw_data['Y']))

19906
19906


In [7]:
print(raw_data['Y'][0][10])

о


In [8]:
symbols_table = {}
codes_table = {}
char_counter = 1
net_data = {}
net_data['X'] = []
net_data['Y'] = []
for sent in raw_data['X']:
    encoded_sentence = []
    for char in sent:
        if char not in symbols_table:
            symbols_table[char] = char_counter
            codes_table[char_counter] = char
            char_counter += 1
        encoded_sentence.append(symbols_table[char])
    encoded_sentence.append(0)
    net_data['X'].append(encoded_sentence)
    
for sent in raw_data['Y']:
    encoded_sentence = []
    for char in sent:
        if char not in symbols_table:
            symbols_table[char] = char_counter
            codes_table[char_counter] = char
            char_counter += 1
        encoded_sentence.append(symbols_table[char])        
    encoded_sentence.append(0)
    net_data['Y'].append(encoded_sentence)

In [9]:
X, Y = [], []
for i in range(len(net_data['X'])):
    if len(net_data['X'][i]) < 256 and len(net_data['Y'][i]) < 256:
        X.append(net_data['X'][i])
        Y.append(net_data['Y'][i])

In [10]:
net_data['X'] = X
net_data['Y'] = Y

In [11]:
max_len = 0
for sent in net_data['X']:
    if len(sent) > max_len:
        max_len = len(sent)
        
for sent in net_data['Y']:
    if len(sent) > max_len:
        max_len = len(sent)
print(max_len)

255


In [12]:
for sent in net_data['X']:
    while len(sent) < max_len:
        sent.append(0)    
        
for sent in net_data['Y']:
    while len(sent) < max_len:
        sent.append(0) 

In [13]:
# for i in range(len(net_data['X'])):
#     net_data['X'][i] = net_data['X'][i][::-1]

In [14]:
print(net_data['X'][0])

[1, 2, 3, 4, 5, 6, 2, 7, 2, 8, 9, 10, 11, 12, 9, 13, 9, 2, 1, 2, 3, 4, 5, 6, 2, 7, 2, 8, 9, 10, 11, 12, 14, 13, 9, 2, 1, 2, 7, 2, 15, 2, 16, 17, 18, 19, 20, 9, 21, 22, 2, 21, 23, 2, 24, 20, 25, 26, 26, 27, 2, 24, 10, 16, 13, 28, 9, 24, 9, 2, 26, 9, 29, 17, 16, 2, 30, 2, 9, 18, 31, 20, 27, 18, 27, 32, 2, 4, 33, 2, 17, 19, 28, 18, 29, 34, 20, 29, 2, 4, 3, 35, 36, 2, 24, 9, 22, 16, 2, 37, 38, 22, 39, 21, 10, 9, 32, 2, 40, 19, 20, 28, 27, 41, 2, 13, 2, 8, 20, 27, 39, 17, 31, 9, 32, 2, 9, 34, 17, 19, 20, 13, 16, 18, 9, 20, 21, 21, 2, 21, 2, 28, 16, 23, 13, 16, 28, 2, 13, 2, 42, 19, 17, 18, 11, 2, 26, 9, 17, 43, 10, 31, 16, 2, 8, 9, 10, 11, 12, 9, 13, 9, 2, 30, 2, 20, 16, 17, 26, 9, 10, 9, 44, 19, 28, 28, 9, 24, 9, 2, 28, 19, 26, 9, 22, 16, 10, 43, 31, 25, 2, 9, 18, 2, 45, 9, 13, 9, 17, 21, 34, 21, 20, 17, 31, 16, 2, 46, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [15]:
net_data['X'] = np.array(net_data['X'])
net_data['Y'] = np.array(net_data['Y'])
Y_shape = np.shape(net_data['Y'])
net_data['Y'] = np.reshape(net_data['Y'],(Y_shape[0],Y_shape[1],1))

In [16]:
data_len = len(net_data['X'])
train_size = int(0.9*data_len)
print(data_len, train_size)

18971 17073


In [17]:
len(symbols_table)

255

In [18]:
from keras.layers import LSTM, Dense, Embedding, TimeDistributed, Activation,RepeatVector, Dropout, Flatten, Permute, Input, merge, Lambda
from keras.models import Sequential, Model
import keras.backend as K

Using TensorFlow backend.


In [40]:
hidden_size = 512 
embedding_size = 10

model2 = Sequential()
model2.add(Embedding(input_dim=char_counter, output_dim=embedding_size, input_length=max_len, mask_zero=True))
model2.add(LSTM(hidden_size))
model2.add(Dropout(0.2))
model2.add(Activation('relu'))
model2.add(RepeatVector(max_len))
model2.add(LSTM(hidden_size, return_sequences=True))
model2.add(Dropout(0.2))
model2.add(TimeDistributed(Dense(char_counter, activation='softmax')))
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',
#              metrics=['accuracy'])

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 255, 10)           2560      
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               1071104   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 255, 512)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 255, 512)          2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 255, 512)          0         
__________

In [19]:
hidden_size = 100 
embedding_size = 10

_input = Input(shape=[max_len], dtype='int32')

# get the embedding layer
embedded = Embedding(
        input_dim=char_counter,
        output_dim=embedding_size,
        input_length=max_len,
        mask_zero=False
    )(_input)

activations = LSTM(hidden_size, return_sequences=True)(embedded)

# compute importance for each step
attention = Dense(1, activation='tanh')(activations)
attention = Flatten()(attention)
attention = Activation('softmax')(attention)
attention = RepeatVector(hidden_size)(attention)
attention = Permute([2, 1])(attention)


sent_representation = merge([activations, attention], mode='mul')
sent_representation = Lambda(lambda xin: K.sum(xin, axis=-2), output_shape=(hidden_size,))(sent_representation)

decoder = LSTM(hidden_size, return_sequences=True)(RepeatVector(max_len)(sent_representation))
probabilities = TimeDistributed(Dense(char_counter, activation='softmax'))(decoder)

model = Model(inputs=[_input], output=probabilities)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

model.summary()

C:\Users\Sdernal\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Users\Sdernal\Anaconda3\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 255)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 255, 10)      2560        input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 255, 100)     44400       embedding_1[0][0]                
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 255, 1)       101         lstm_1[0][0]                     
__________________________________________________________________________________________________
flatten_1 

C:\Users\Sdernal\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("ti...)`


In [38]:
model.fit(net_data['X'][:train_size], net_data['Y'][:train_size],batch_size=1024, epochs=5, validation_data=(net_data['X'][train_size:], net_data['Y'][train_size:]))

Train on 17073 samples, validate on 1898 samples
Epoch 1/5
17073/17073 [==============================] - 16s 952us/step - loss: 0.9991 - acc: 0.7542 - val_loss: 1.4477 - val_acc: 0.6401
Epoch 2/5
17073/17073 [==============================] - 16s 951us/step - loss: 0.9947 - acc: 0.7555 - val_loss: 1.4432 - val_acc: 0.6407
Epoch 3/5
17073/17073 [==============================] - 17s 981us/step - loss: 0.9936 - acc: 0.7556 - val_loss: 1.4432 - val_acc: 0.6408
Epoch 4/5
17073/17073 [==============================] - 16s 940us/step - loss: 0.9932 - acc: 0.7555 - val_loss: 1.4436 - val_acc: 0.6408
Epoch 5/5
17073/17073 [==============================] - 16s 938us/step - loss: 0.9931 - acc: 0.7558 - val_loss: 1.4422 - val_acc: 0.6408


In [46]:
net_answer = model.predict(net_data['X'][1000:1010])

In [22]:
net_data['X'][0]

array([ 1,  2,  3,  4,  5,  6,  2,  7,  2,  8,  9, 10, 11, 12,  9, 13,  9,
        2,  1,  2,  3,  4,  5,  6,  2,  7,  2,  8,  9, 10, 11, 12, 14, 13,
        9,  2,  1,  2,  7,  2, 15,  2, 16, 17, 18, 19, 20,  9, 21, 22,  2,
       21, 23,  2, 24, 20, 25, 26, 26, 27,  2, 24, 10, 16, 13, 28,  9, 24,
        9,  2, 26,  9, 29, 17, 16,  2, 30,  2,  9, 18, 31, 20, 27, 18, 27,
       32,  2,  4, 33,  2, 17, 19, 28, 18, 29, 34, 20, 29,  2,  4,  3, 35,
       36,  2, 24,  9, 22, 16,  2, 37, 38, 22, 39, 21, 10,  9, 32,  2, 40,
       19, 20, 28, 27, 41,  2, 13,  2,  8, 20, 27, 39, 17, 31,  9, 32,  2,
        9, 34, 17, 19, 20, 13, 16, 18,  9, 20, 21, 21,  2, 21,  2, 28, 16,
       23, 13, 16, 28,  2, 13,  2, 42, 19, 17, 18, 11,  2, 26,  9, 17, 43,
       10, 31, 16,  2,  8,  9, 10, 11, 12,  9, 13,  9,  2, 30,  2, 20, 16,
       17, 26,  9, 10,  9, 44, 19, 28, 28,  9, 24,  9,  2, 28, 19, 26,  9,
       22, 16, 10, 43, 31, 25,  2,  9, 18,  2, 45,  9, 13,  9, 17, 21, 34,
       21, 20, 17, 31, 16

In [39]:
np.reshape(net_data['Y'],(Y_shape[0],Y_shape[1]))[0]

array([ 1,  2,  3,  4,  5,  6,  2,  7,  2, 31,  9, 10, 11, 12,  9, 13,  9,
        2,  1,  2,  3,  4,  5,  6,  2,  7,  2, 31,  9, 10, 11, 12, 14, 13,
       27, 32,  2,  1,  2,  7,  2, 15,  2, 16, 17, 18, 19, 20,  9, 21, 22,
        2, 21, 23,  2, 24, 20, 25, 26, 26, 16,  2, 24, 10, 16, 13, 28, 27,
       32,  2, 26,  9, 29, 17,  2, 30,  2,  9, 18, 31, 20, 27, 18, 27, 32,
        2,  4, 33,  2, 17, 19, 28, 18, 29, 34, 20, 11,  2,  4,  3, 35, 36,
        2, 24,  9, 22,  2, 10, 38, 22, 39, 21, 10, 16,  2, 42, 43, 20, 28,
       27, 32,  2, 13,  2, 31, 20, 27, 39, 17, 31, 21, 32,  2,  9, 34, 17,
       19, 20, 13, 16, 18,  9, 20, 21, 29,  2, 21,  2, 28, 16, 23, 13, 16,
       18, 11,  2, 13,  2, 42, 19, 17, 18, 11,  2, 26,  9, 17, 43, 10,  9,
       31,  2, 31,  9, 10, 11, 12,  9, 13,  9,  2, 30,  2, 20, 16, 17, 26,
        9, 10,  9, 44, 21, 18, 11,  2, 28, 19, 26,  9, 22, 16, 10, 43, 31,
       25,  2,  9, 18,  2, 28,  9, 13,  9, 17, 21, 34, 21, 20, 17, 31,  2,
       46,  0,  0,  0,  0

In [47]:
answer_sents = []
for sent in net_answer:
    answer_sent = ''
    for char in sent:
        res = np.argmax(char)        
        if res > 0:
            letter = codes_table[res]
            answer_sent += codes_table[res]
    answer_sents.append(answer_sent)

In [48]:
for sent in answer_sents:
    print(sent)

: `` г  '''' аниитерр..
: `` '' сантимее  
: ``
: `` г  '''' аниитетр..
: `` '' сантиметр .
: ``
: `` гд '''' анииметр..
: `` '' сантиметр .
: ``
: `` г  '''' аниитетр..


In [49]:
for i in range(1000,1010):
    print(raw_data['X'][i])

1216 год '' См .
`` '' См .
``
1217 год '' См .
: `` '' См .
: ``
1218 год '' См .
: `` '' См .
: ``
1219 год '' См .
